In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import re
import copy

In [2]:
path = 'D:/AS/'

In [3]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [4]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']=='AS']

In [5]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')

In [6]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')

In [7]:
vt_tax_based_on = pd.read_csv(r'D:\AS\vt_tax_based_on.csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\AS\vt_tax_based_on_permit_info.csv')

In [8]:
vm_vh_class=pd.read_csv(r'D:\Jharkhand python\vm_vh_class.csv')

In [9]:
vt_owner = pd.read_csv(r'D:\AS\vt_owner.csv')

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
vt_permit = pd.read_csv(r'D:\AS\vt_permit.csv')

In [11]:
tm_office = pd.read_csv(r'D:\AS\tm_office.csv')

In [13]:
vt_tax_exem = pd.read_csv(r'D:\AS\vt_tax_exem.csv')

In [12]:
vt_tax = pd.read_csv(r'D:\AS\vt_tax.csv')
vt_tax = vt_tax[vt_tax['Pur Cd']==58]
vt_tax = vt_tax[vt_tax['Tax Mode']!='B']
#vt_tax=vt_tax[vt_tax['Collected By']!='VAHAN2']
vt_tax.reset_index(drop=True, inplace=True)

In [11]:
t_cols =['rcpt_no','rcpt_dt','regn_no','tax_from','tax_upto','off_cd']
tc_cols = ['off_cd','appl_no','regn_no','clear_fr','clear_to','iss_dt']
te_cols = ['off_cd','regn_no','exem_fr','exem_to','perm_no','perm_dt']

In [13]:
#Turn into first letter capital in each word
def change_col_names(cols):
    pp=[]
    for j in cols :
        p= j.replace('_',' ').title()
        pp.append(p)
        #df.rename(columns={j: p},inplace=True)
    return pp

In [13]:
t_cols = change_col_names(t_cols)
tc_cols = change_col_names(tc_cols)
te_cols = change_col_names(te_cols)

In [14]:
df1=vt_tax.loc[:,t_cols]
df2=vt_tax_clear.loc[:,tc_cols]
df3 = vt_tax_exem.loc[:,te_cols]

NameError: name 'vt_tax_clear' is not defined

In [19]:
df1 = df1.rename(columns = {'Tax From':'From','Tax Upto':'To','Rcpt Dt': 'Issue Dt'})
df2 = df2.rename(columns={'Clear Fr': 'From','Clear To':'To','Iss Dt':'Issue Dt','Appl No':'Rcpt No'})
df3 = df3.rename(columns={ 'Exem Fr':'From','Exem To':'To','Perm No':'Rcpt No','Perm Dt':'Issue Dt'})

In [20]:
df1['Table'] = 'vt_tax'
df2['Table'] = 'vt_tax_clear'
df3['Table'] = 'vt_tax_exem'

In [21]:
frames = [df1,df2,df3]
df = pd.concat(frames)

In [24]:
df = df[df['Off Cd']==1]

In [25]:
df.count()

Rcpt No     1307147
Issue Dt     622814
Regn No     1307180
From        1307180
To          1307171
Off Cd      1307180
Table       1307180
dtype: int64

In [28]:
df['Table'].value_counts()

vt_tax_clear    795355
vt_tax          497788
vt_tax_exem      14037
Name: Table, dtype: int64

In [27]:
dups = df[df.duplicated(['Rcpt No'], keep='first')]
dups.shape

(3771, 7)

In [ ]:
def_regn_dt = []
def_purchase_dt = []
def_regn_no = []
def_vh_class = []
def_latest_tax_paid_on = []
def_latest_tax_paid_from = []
def_latest_tax_paid_to = []
def_latest_tax_paid_rcpt_no = []
def_latest_table = []
def_state_cd = []
def_off_cd = []
p = 1
q=0
for i in set(df['Regn No'].to_list()):
    q=q+1
    vh_class = vt_owner[vt_owner['Regn No']==i]['Vh Class'].to_list()
    if len(vh_class)==0:
        print('No VH_CLASS AVAILABLE for ',i)
    else:
        vh_class = vh_class[0]
        if vm_vh_class[vm_vh_class['Vh Class']==vh_class]['Class Type'].to_list()[0] == 1:   
            regn_no_hist = df[df['Regn No']==i]   
            regn_no_hist.reset_index(drop=True, inplace=True)
            upto_list = regn_no_hist['To'].astype('datetime64[ns]').to_list()
            latest_tax_paid_upto = max(upto_list)
            ind = upto_list.index(max(upto_list))
            if latest_tax_paid_upto is None:
                print('Tax paid upto date is Null')
            else:
                #ind = max(df[(df['Regn No']==i)&(df['To']==latest_tax_paid_upto)].index.to_list())
                x = relativedelta(datetime.strptime('2019-10-03','%Y-%m-%d'),datetime.strptime(str(latest_tax_paid_upto)[:10], '%Y-%m-%d'))
                x = x.years*365 + x.months*30 + x.days
                if x>0:
                    print(q,p,i,latest_tax_paid_upto)
                    p=p+1
                    def_regn_no.append(i)
                    def_regn_dt = vt_owner[vt_owner['Regn No']==i]['Regn Dt'].to_list()[0]
                    def_purchase_dt = vt_owner[vt_owner['Regn No']==i]['Purchase Dt'].to_list()[0]
                    #def_state_cd = df.loc[ind,'State Cd']
                    def_off_cd = regn_no_hist.loc[ind,'Off Cd']
                    def_vh_class = vh_class
                    def_latest_tax_paid_on = regn_no_hist.loc[ind,'Issue Dt']
                    def_latest_tax_paid_from = regn_no_hist.loc[ind,'From']
                    def_latest_tax_paid_to = latest_tax_paid_upto
                    def_latest_tax_paid_rcpt_no = regn_no_hist.loc[ind,'Rcpt No']
                    def_latest_table = regn_no_hist.loc[ind,'Table']
def_list = pd.DataFrame(list(zip(def_regn_dt, def_purchase_dt, def_regn_no,def_state_cd, def_off_cd, def_vh_class, def_latest_tax_paid_on, def_latest_tax_paid_from, def_latest_tax_paid_to, def_latest_tax_paid_rcpt_no,def_latest_table)))



4 1 JH01BL0873 2019-09-11 00:00:00
8 2 JH01P2457 2007-05-18 00:00:00
22 3 BR14P5401 2018-07-20 00:00:00
25 4 JH01A5864 2006-12-23 00:00:00
No VH_CLASS AVAILABLE for  ZZ15T58661
31 5 JH01T2690 2019-02-02 00:00:00
41 6 JH01CS6421 2018-05-27 00:00:00
58 7 JH01T7536 2012-01-05 00:00:00
105 8 JH01BZ2309 2016-08-26 00:00:00
132 9 JH01Y3338 2011-01-07 00:00:00
145 10 JH01CF0608 2019-07-20 00:00:00
156 11 BPN8961 2007-08-06 00:00:00
164 12 BR14D5941 2007-02-01 00:00:00
No VH_CLASS AVAILABLE for  JH01BW6946
168 13 JH01N4785 2006-05-20 00:00:00
No VH_CLASS AVAILABLE for  JH01DQ4336
174 14 JH01BP6838 2015-07-16 00:00:00
176 15 JH01BF2935 2015-03-20 00:00:00
178 16 JH01AQ4428 2018-09-19 00:00:00
193 17 JH01AZ9507 2014-06-12 00:00:00
194 18 JH01AC4014 2011-11-26 00:00:00
205 19 JH01Q3444 2008-12-25 00:00:00
219 20 JH01CW2134 2018-10-11 00:00:00
No VH_CLASS AVAILABLE for  JH/1/TMP/2018/3559
248 21 JH01CF5022 2017-10-23 00:00:00
No VH_CLASS AVAILABLE for  JH01DP4661
260 22 JH01T0681 2007-10-14 00:00:

No VH_CLASS AVAILABLE for  JH01DQ8469
1962 179 JH01L4699 2010-09-08 00:00:00
1964 180 BR14G0284 2015-11-04 00:00:00
No VH_CLASS AVAILABLE for  JH01DQ4075
2010 181 JH01Y5920 2013-08-02 00:00:00
2011 182 JH01U5303 2017-12-31 00:00:00
2013 183 JH01T1564 2012-07-25 00:00:00
2020 184 JH01E6672 2009-12-05 00:00:00
2036 185 JH01BG3565 2015-11-23 00:00:00
2038 186 JH01V0804 2011-02-22 00:00:00
2043 187 JH01N8700 2009-07-06 00:00:00
No VH_CLASS AVAILABLE for  JH01DQ7897
2066 188 UP61B7017 2008-03-31 00:00:00
2068 189 JH01K1818 2011-04-03 00:00:00
No VH_CLASS AVAILABLE for  JH/1/TMP/2019/1346
2084 190 JH01AQ3245 2018-06-12 00:00:00
2094 191 BR1P2098 2011-08-11 00:00:00
2099 192 JH01BH7275 2016-11-01 00:00:00
2126 193 JH01CS0310 2018-10-23 00:00:00
2137 194 JH01Q8617 2009-07-19 00:00:00
2157 195 JH01AE0358 2011-03-11 00:00:00
2191 196 JH01CF5998 2018-03-31 00:00:00
2196 197 JH01C0189 2019-05-07 00:00:00
2203 198 JH01G3954 2013-04-26 00:00:00
2221 199 JH01AQ9209 2017-07-19 00:00:00
2239 200 BHM607

4067 349 JH01CF8925 2018-05-27 00:00:00
4071 350 JH01AP6991 2018-11-18 00:00:00
4076 351 BR14P6595 2013-10-16 00:00:00
4081 352 JH01AD7457 2010-05-20 00:00:00
4120 353 JH01CY6074 2019-03-03 00:00:00
4122 354 JH01AQ7058 2012-11-15 00:00:00
4134 355 JH01BZ5433 2016-06-29 00:00:00
4143 356 JH01S7279 2007-09-10 00:00:00
4181 357 JH01P5843 2008-06-28 00:00:00
No VH_CLASS AVAILABLE for  JH/1/TMP/2018/4208
4196 358 JH01AM2550 2011-08-30 00:00:00
No VH_CLASS AVAILABLE for  JH/1/TMP/2019/2243
4226 359 JH01AL9209 2012-02-13 00:00:00
No VH_CLASS AVAILABLE for  JH01DQ2348
4229 360 JH01BH7965 2015-07-14 00:00:00
No VH_CLASS AVAILABLE for  JH/1/TMP/2019/2179
4234 361 JH01AW9581 2017-12-26 00:00:00
4236 362 JH01BB4461 2017-09-25 00:00:00
4241 363 JH01AE2275 2011-03-25 00:00:00
4248 364 JH01CH2171 2018-03-03 00:00:00
4264 365 JH01Q6359 2012-08-01 00:00:00
4273 366 BR13G0513 2011-03-23 00:00:00
4281 367 BR05G1831 2016-10-24 00:00:00
4283 368 BR14F2364 2011-03-25 00:00:00
4292 369 JH01BM0435 2017-10-07 

In [143]:
len(set(df['Regn No'].to_list()))

1039719

In [20]:
def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Kamrup DTO Defaulter list up to March 2018 (1).xls')
def_list.columns

Index(['S. No', 'Regn No', 'Office', 'Regn Date', 'Vehicle Class',
       'Vehicle Category', 'Tax Type', 'Owner Name', 'Father Name',
       'Complete Present Address', 'Mobile No', 'Laden Wt.', 'UnLaden Wt.',
       'Seating Capacity', 'Standing Capacity', 'Fuel Type', 'Tax Arrear From',
       'Tax Arrear UpTo', 'Permit Valid From', 'Permit Valid Upto',
       'Tax Amount in (Rs)', 'Tax Fine in (Rs)', 'Total Tax Amount in (Rs)',
       'Fitness Validity', 'Financer Name'],
      dtype='object')

In [14]:
vt_tax_based_on['op_dt'].head

<bound method NDFrame.head of 0          08-02-2017 08:52:18
1          01-01-2019 17:58:21
2          20-07-2019 19:18:36
3          29-03-2017 13:16:45
4          09-11-2017 17:57:17
                  ...         
1778002    02-10-2019 19:10:41
1778003    02-10-2019 19:10:14
1778004    29-08-2019 16:13:22
1778005    02-10-2019 19:13:46
1778006    02-10-2019 19:14:19
Name: op_dt, Length: 1778007, dtype: object>

In [14]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
        else:
            c_formula = c_formula.replace('<>','!=')

    # Change '=' to '=='
    if '<=' in c_formula:
        c_formula=c_formula.replace('<=','@')
    if '>=' in c_formula:
        c_formula=c_formula.replace('>=',';')
    if '!=' in c_formula:
        c_formula=c_formula.replace('!=','?')
    if '=' in c_formula:
        c_formula = c_formula.replace('=','==')
    if '@' in c_formula:
        c_formula=c_formula.replace('@','<=')
    if ';' in c_formula:
        c_formula=c_formula.replace(';','>=')
    if '?' in c_formula:
        c_formula = c_formula.replace('?','!=')
    return c_formula

In [110]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%d-%m-%Y'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [111]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,new_vch,rcpt_dt):
    for pi in range(len(param_info_owner)):
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        col= col.replace('_',' ').title()
        p_value = ''
        #print(col,par,code,tab)
        if par=='vch_age':
            p_value = relativedelta(rcpt_dt, datetime.strptime(ot_df['Purchase Dt'].to_list()[0],date_format))
            p_value = p_value.years
        elif par=='vch_age_months':
            p_value = relativedelta(rcpt_dt, datetime.strptime(ot_df['Purchase Dt'].to_list()[0],date_format))
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age':
            p_value = relativedelta(rcpt_dt, datetime.strptime(ot_df['Purchase Dt'].to_list()[0],date_format))
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        #elif par=='distance_run_in_month':
        #    p_value = vt_tax_based_on['distance_run_in_quarter']/4
        #elif par=='transaction_pur_cd':
        #    appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
        #    if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
        #        print(i,'No pur_cd available in va_details')
        #        print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        #        continue
        #    else:
        #        p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
        #print(p_value)
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['vh_class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = new_vch
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='delay_days':
            #print(rcpt_dt[:10],' ',tax_from)
            p_value = rcpt_dt-tax_from
            #p_value = p_value.years * 365 + p_value.months * 30 + p_value.days
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
            #print(p_value)
        else:
            if tab=='own':
                col= col.replace('_',' ').title()
            p_value= ot_df[col].to_list()[0]
            #print(p_value)
        #print(col,' ',par,' ',code,' ',p_value)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
        #print(code,p_value)
    return params_info

In [112]:
def extract_parameters(regn_no,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['Regn No']==regn_no]
        owner_df= copy.deepcopy(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            for j in c :
                p= j.replace('_',' ').title()
                tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]

    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        #print('TBO')
        tab = 'tbo'
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,new_vch,rcpt_dt)
        #print('TBO',params_info['P_value'].to_list())
    else:
        #print('Owner')
        tab='own'
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,new_vch,rcpt_dt)
    #print('Shape',params_info.shape)
    if len(param_info_other)>0:  
        #print('Other')
        #print(param_info_other.shape)
        for pi in range(len(param_info_other)):
            col = param_info_other['column_name'].to_list()[pi][1:-2]
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            #print(code,col,par)
            
            #print(null_df)
            if math.isnan(params_info[params_info['code']==code]['P_value'].to_list()[0]):
                if col == 'domain_cd':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Domain Cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        print('No Domain Cd Available')
                elif col == 'pmt_type':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Pmt Type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list()[0]
                    else:
                        p_value = None
                        print('No Permit Type Available')
                elif col== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                #print(p_value)

                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value  
    return params_info

In [113]:
date_format = "%d-%b-%Y"
def calculate_tax(regn_no,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Q' ,state_cd='AS',off_cd=1):
    rcpt_dt = tax_upto
    if len(vt_owner[vt_owner['Regn No']==regn_no])>0:
        vh_class = vt_owner[vt_owner['Regn No']==regn_no]['Vh Class'].to_list()[0]
        new_vch = 'N'
        #print(vh_class)
    else:
        print(i,'NO INFO IN VT_OWNER',regn_no)
        
    slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<tax_from) & (slab_new['date_to']>tax_from) ]['slab_code'].to_list()
    #print(pur_cd,'Slab Codes Applicable:',slab_codes)
    if len(slab_codes)==0:
        print(i,'Missed',regn_no,'vh_class',vh_class)
        return 0

    formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
        
    #Identify the parameters required for calculating the formulas
    params = []
    for j in range(len(formula_df)):
        formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
        for k in re.split(r'[()\*\s]\s*',formula):
            if '<' in k and '>' in k:
                params.append(k[k.find('<'):(k.find('>')+1)])
    params = list(set(params))
    #print('Required Parameters :',params)
    params_info = slab_fields.loc[slab_fields['code'].isin(params)]
    params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
    #print(params_info)
    param_info_owner = params_info[params_info['def_table']=='vt_owner']
    #print('Owner',param_info_owner['column - slab_field'].to_list())
    param_info_other = params_info[params_info['def_table']!='vt_owner']
    #print('Other',param_info_other['column - slab_field'].to_list())
    #param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
    #param_info_misc = params_info[(params_info['table - slab_field']!='vt_tax_based_on_permit_info') & (params_info['table - slab_field']!='vt_tax_based_on')]
    param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
    #print('TBO',param_info_tbo['code'].to_list)
    #print('TBO',param_info_tbo['column - slab_field'].to_list())
    #print('Initial Shape',params_info.shape)
    params_info = extract_parameters(regn_no,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd)
    #print('Final Shape',params_info.shape)                                                                        
    #if pur_cd ==57:        
    #print(params_info)
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R':
            #print(c_formula)
            cf_result = eval(c_formula.lower())
        else:
            cf_result = False
        if cf_result==True:
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    #print(summary_df)
    if len(ind)==0:
        print('No True Condition')
        return 
    #Module for calculation of rates for true conditions
    true_df = formula_df.loc[ind,:]
    true_df.reset_index(drop=True, inplace=True)
    #print(true_df)
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print(t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        #print(r_formula)
        if 'CEILING' in r_formula: r_formula.replace('CEIL','math.ceil')
        if 'CEIL' in r_formula: r_formula = r_formula.replace('CEIL','math.ceil')
        rate = eval(r_formula.lower())
        rates.append(rate)
        
    true_df['Rate']=rates
    #print(rates)
    #print(true_df)
    #Module for dealing Minimum M and Amount T 
    if len(set(true_df['condition_formula'].to_list()))>1:
        #print(True)
        true_df = true_df[true_df['condition_formula']!='1=1']
        if len({'T','M'} - set(true_df['type_flag'].to_list()))==0:
            if true_df[true_df['type_flag']=='T']['Rate'].to_list()[0]> true_df[true_df['type_flag']=='M']['Rate'].to_list()[0]:
                final_rate = true_df[true_df['type_flag']=='T']['Rate'].to_list()
            else:
                final_rate = true_df[true_df['type_flag']=='M']['Rate'].to_list()
    else:       
        final_rate = true_df['Rate'].to_list()
    if len(final_rate)>0:
        final_rate = final_rate[0]
    else:
        final_rate = 0
        #print(final_rate)
    '''     
    #Module for Tax Mode Remaining years
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        #print('Tax Upto - Tax From =',tax_period,'days')
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        #print(((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days))
        if tax_mode=='Y10':
            days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/3652)*days_rem
        if tax_mode=='Y12':
            days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            #print('Days remaining in ',tax_mode,'=',days_rem,'days')
            final_rate = (final_rate/4383)*days_rem
        if tax_mode=='Y15':
            days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode    
    if tax_mode== 'Q':
        #print(tax_upto,' ',tax_from)
        tax_period = relativedelta(tax_upto, tax_from)
        #print(tax_period)
        years_q = tax_period.years * 4 
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        #print(n_quart)
        final_rate = final_rate * n_quart

    #Display Calculated Amount
    return final_rate

In [114]:
nc_regn_no_list = []
for i in range(23,25):   
    regn_no = def_list['Regn No'].to_list()[i]
    print(regn_no)
    #state_cd = 'AS'
    #off_cd = 1
    #rcpt_no = def_list['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    #tax_amt = def_list['Tax Amt'].to_list()[i]
    #rcpt_dt = def_list['Rcpt Dt'].to_list()[i]
    
    #tax_from = def_list['latest_tax_paid_upto'] + 1
    tax_from = def_list['Tax Arrear From'].to_list()[i]
    tax_from = datetime.strptime(tax_from, date_format)
    #print(tax_from)
    tax_upto = def_list['Tax Arrear UpTo'].to_list()[i]
    tax_upto = datetime.strptime(tax_upto,'%d-%b-%Y')
    #print(tax_upto)
    c_tax_amt = calculate_tax(regn_no,tax_from,tax_upto)
    if c_tax_amt == 0:
        print('Unable to calculate tax')
        nc_regn_no_list.append(regn_no)
    tax_amt = pd.to_numeric(def_list['Tax Amount in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    
    tax_fine = pd.to_numeric(def_list['Tax Fine in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    if c_tax_amt !=0:
        c_tax_fine = calculate_tax(regn_no,tax_from,tax_upto, pur_cd = 57, final_rate = c_tax_amt,tax_mode ='Y')
    else:
        c_tax_fine = 0
    #print(tax_fine)
    if tax_amt!=c_tax_amt: 
        print('Mismatch',i,regn_no,tax_amt,c_tax_amt,c_tax_fine,tax_fine)
    else:
        print(i, regn_no, tax_amt, c_tax_amt,c_tax_fine,tax_fine)
    #summary_df = c_tax_amt[1]

AS01HC6923
23 AS01HC6923 4200.0 4200 3190 2745.0
AS01U6419
Mismatch 24 AS01U6419 5234.0 5400 3975 2155.0


In [82]:
math.isnan(slab_fields[slab_fields['code']=='<61>']['def_table'].to_list()[0])

True